# EVALUATION MODULE MongoDB
### Mohamed TOUMI

=================

### Connexion à la base de données

(a) Pour se connecter à MongoDB, lancer la cellule suivante:

In [2]:
from pymongo import MongoClient

full_dns_name = 'mongodb://ec2-54-229-30-76.eu-west-1.compute.amazonaws.com:27017'
username = 'test'
password = 'test'
authSource = 'admin'

client = MongoClient(host=full_dns_name, username=username, password=password, authSource=authSource)

(b) afficher la liste des bases de données disponibles

In [3]:
print(client.list_database_names())

['arxiv']


(c) afficher la liste des collections disponibles dans cette base de données

In [4]:
arxiv = client["arxiv"]
print(arxiv.list_collection_names())

['papers']


(d) afficher un des documents de cette collection

In [5]:
from pprint import pprint 

papers = arxiv["papers"]
pprint(papers.find_one())

{'_id': ObjectId('5fa9a4db76fdd8d66273c643'),
 'abstract': '  A fully differential calculation in perturbative quantum '
             'chromodynamics is\n'
             'presented for the production of massive photon pairs at hadron '
             'colliders. All\n'
             'next-to-leading order perturbative contributions from '
             'quark-antiquark,\n'
             'gluon-(anti)quark, and gluon-gluon subprocesses are included, as '
             'well as\n'
             'all-orders resummation of initial-state gluon radiation valid '
             'at\n'
             'next-to-next-to-leading logarithmic accuracy. The region of '
             'phase space is\n'
             'specified in which the calculation is most reliable. Good '
             'agreement is\n'
             'demonstrated with data from the Fermilab Tevatron, and '
             'predictions are made for\n'
             'more detailed tests with CDF and DO data. Predictions are shown '
             'for\n'

(e) afficher le nombre de documents dans cette collection

In [7]:
print(papers.count_documents({}))

10000


### Exploration de la base

(a) afficher le titre de 10 articles

In [10]:
print(list(papers.find({}, {"_id":0, "title":1}).limit(10)))

[{'title': 'Calculation of prompt diphoton production cross sections at Tevatron and\n  LHC energies'}, {'title': 'Sparsity-certifying Graph Decompositions'}, {'title': 'The evolution of the Earth-Moon system based on the dark matter field\n  fluid model'}, {'title': 'A determinant of Stirling cycle numbers counts unlabeled acyclic\n  single-source automata'}, {'title': 'From dyadic $\\Lambda_{\\alpha}$ to $\\Lambda_{\\alpha}$'}, {'title': 'Bosonic characters of atomic Cooper pairs across resonance'}, {'title': 'Polymer Quantum Mechanics and its Continuum Limit'}, {'title': 'Numerical solution of shock and ramp compression for general material\n  properties'}, {'title': 'The Spitzer c2d Survey of Large, Nearby, Insterstellar Clouds. IX. The\n  Serpens YSO Population As Observed With IRAC and MIPS'}, {'title': 'Partial cubes: structures, characterizations, and constructions'}]


(b) afficher les 10 premiers auteurs par ordre alphabétique

In [127]:
print(list(papers.aggregate([{"$sort":{"authors":1}}, {"$project":{"_id":0, "authors":1}}]))[0:9])

[{'authors': '(The TeV4LHC-Top and Electroweak Working Group:) C.E.Gerber, P.Murat,\n  T.M.P.Tait, D.Wackeroth, A.Arbuzov, D.Bardin, U.Baur, J.A.Benitez, S.Berge,\n  S.Bondarenko, E.E.Boos, M.T.Bowen, R.Brock, V.E.Bunichev, J.Campbell,\n  F.Canelli, Q.-H.Cao, C.M.Carloni Calame, F.Chevallier, P.Christova,\n  C.Ciobanu, S.Dittmaier, L.V.Dudko, S.D.Ellis, A.I.Etienvre, F.Fiedler,\n  A.Garcia-Bellido, A.Giammanco, D.Glenzinski, P.Golonka, C.Hays, S.Jadach,\n  S.Jain, L.Kalinovskaya, M.Kramer, A.Lleres, J.Luck, A.Lucotte, A.Markina,\n  G.Montagna, P.M.Nadolsky, O.Nicrosini, F.I.Olness, W.Placzek, R.Sadykov,\n  V.I.Savrin, R.Schwienhorst, A.V.Sherstnev, S.Slabospitsky, B.Stelzer,\n  M.J.Strassler, Z.Sullivan, F.Tramontano, A.Vicini, W.Wagner, Z.Was, G.Watts,\n  M.Weber, S.Willenbrock, U.K.Yang, C-P.Yuan, J.Zhu'}, {'authors': 'A Brandenburg (Nordita) and P J K\\"apyl\\"a (Nordita and Helsinki\n  Observatory)'}, {'authors': 'A Kuiroukidis, K Kleidis and D B Papadopoulos'}, {'authors': 'A Kuir

(c) afficher le nombre d'articles qui n'ont pas été publiés par "Damien Chablat"

In [135]:
print(papers.count_documents({"submitter":{"$ne":"Damien Chablat"}}))

9968


(d) afficher le nombre de papiers téléversés en 2014. On pourra s'appuyer sur une expréssion régulière.

In [143]:
import re

exp = re.compile(r"2014-\d\d-\d\d")
print(papers.count_documents({"update_date":exp}))

122


(e) afficher le titre d'un article qui contient l'expression Machine Learning. On pourra utiliser une expression régulière.

In [155]:
exp2 = re.compile(r"Machine+ Learning+")
print(list(papers.find({"abstract":exp2}, {"_id":0, "title":1, "abstract":1})))

[{'title': 'Ensemble Learning for Free with Evolutionary Algorithms ?', 'abstract': '  Evolutionary Learning proceeds by evolving a population of classifiers, from\nwhich it generally returns (with some notable exceptions) the single\nbest-of-run classifier as final result. In the meanwhile, Ensemble Learning,\none of the most efficient approaches in supervised Machine Learning for the\nlast decade, proceeds by building a population of diverse classifiers. Ensemble\nLearning with Evolutionary Computation thus receives increasing attention. The\nEvolutionary Ensemble Learning (EEL) approach presented in this paper features\ntwo contributions. First, a new fitness function, inspired by co-evolution and\nenforcing the classifier diversity, is presented. Further, a new selection\ncriterion based on the classification margin is proposed. This criterion is\nused to extract the classifier ensemble from the final population only\n(Off-line) or incrementally along evolution (On-line). Experimen

(f) afficher le nombre de publications par submitter pour les 10 personnes les plus prolifiques. On pourra utiliser un pipeline d'aggrégation avec les mots-clefs $group, $sort, $limit.

In [164]:
print(list(papers.aggregate([
    {
        "$group":{"_id":"$submitter", "nb_paper":{"$sum":1}}
    },
    {
        "$sort":{"nb_paper":-1}
    },
    {
        "$limit":10
    }
])
))

[{'_id': 'Tshilidzi Marwala', 'nb_paper': 36}, {'_id': 'Damien Chablat', 'nb_paper': 32}, {'_id': 'Yusuke Sasano', 'nb_paper': 18}, {'_id': 'Jerome Darmont', 'nb_paper': 12}, {'_id': 'Matthias Brust R.', 'nb_paper': 11}, {'_id': 'Laurent Cognet', 'nb_paper': 10}, {'_id': 'Dr. Anirudh Pradhan', 'nb_paper': 9}, {'_id': 'Michael Ibison', 'nb_paper': 8}, {'_id': 'Dietrich Stauffer', 'nb_paper': 8}, {'_id': 'Gordan Zitkovic', 'nb_paper': 8}]
